# Parent Document Retriever
When splitting documents for retrieval, there are often conflicting desires:

1. You may want to have small documents, so that their embeddings can most accurately reflect their meaning. If too long, then the embeddings can lose meaning.
2. You want to have long enough documents that the context of each chunk is retained.

The `ParentDocumentRetriever` strikes that balance by splitting and storing small chunks of data. During retrieval, it first fetches the small chunks but then looks up the parent ids for those chunks and returns those larger documents.

Note that “parent document” refers to the document that a small chunk originated from. This can either be the whole raw document OR a larger chunk.

In [1]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import SentenceTransformerEmbeddings


In [2]:
from IPython.display import display, Markdown
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader1 = PyPDFLoader("./data/Raptor Contract.docx.pdf")
loader2 = PyPDFLoader("./data/Robinson Advisory.docx.pdf")

loaders = [loader1, loader2]
docs =[]

for loader in loaders:
    docs.extend(loader.load())

In [4]:
# from langchain_community.document_loaders import DirectoryLoader

# loader = DirectoryLoader("./data/Raptor Contract.docx.pdf")
# docs = loader.load()

In [5]:

loader_test = PyPDFLoader("./data/Robinson Advisory.docx.pdf")
docs_test = loader_test.load()
docs_test

[Document(page_content='ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdvisory\nServices\nAgreement\nis\nentered\ninto\nas\nof\nJune\n15\nth\n,\n2023\n(the\n“\nEffective\nDate\n”),\nby\nand \nbetween\nCloud\nInvestments\nLtd.,\nID\n51-426526-3,\nan\nIsraeli\ncompany\n(the\n"\nCompany\n"),\nand\nMr.\nJack \nRobinson,\nPassport\nNumber\n780055578,\nresiding\nat\n1\nRabin\nst,\nTel\nAviv,\nIsrael,\nEmail: \njackrobinson@gmail.com\n("\nAdvisor\n").\nWhereas,\nAdvisor\nhas\nexpertise\nand/or\nknowledge\nand/or\nrelationships,\nwhich\nare\nrelevant\nto\nthe \nCompany’ s\nbusiness\nand\nthe\nCompany\nhas\nasked\nAdvisor\nto\nprovide\nit\nwith\ncertain\nAdvisory \nservices,\nas\ndescribed\nin\nthis\nAgreement;\nand\nWhereas,\nAdvisor\nhas\nagreed\nto\nprovide\nthe\nCompany\nwith\nsuch\nservices,\nsubject\nto\nthe\nterms\nset\nforth \nin\nthis\nAgreement.\nNOW\nTHEREFORE\nTHE\nPARTIES\nAGREE\nAS\nFOLLOWS:\n1.\nServices:\n1.1\nAdvisor\nshall\nprovide\nto\nthe\nCompany ,\nas\nan\nindependent\ncontractor

# Retrieving full documents
In this mode, we want to retrieve the full documents. Therefore, we only specify a child splitter.

In [6]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

st_embedder = SentenceTransformerEmbeddings()
openai_embedder = OpenAIEmbeddings()

/home/crs/10Academy/w11/Contract-Advisor-RAG/w11_rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=SentenceTransformerEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [8]:
retriever.add_documents(docs_test, ids=None)

This should yield two keys, because we added two documents. However, this is not the case because we use PyPDFLoader rather DirectoryLoader.

In [9]:
list(store.yield_keys())

['7945f33c-d3ec-406b-87e4-50e92fea56e8',
 'f578361e-57e2-4f4f-b62f-cb0a99c506ec',
 '49ef6d00-8881-46bd-8b34-8d4a1f6cd85a',
 '9dfcb5d8-9e21-449f-8453-d648abfd07dd']

Let’s now call the vector store search functionality - we should see that it returns small chunks (since we’re storing the small chunks).

In [10]:
sub_docs = vectorstore.similarity_search("In which street does the Advisor live?")

In [11]:
sub_docs

[Document(page_content='(the\n"\nCompany\n"),\nand\nMr.\nJack \nRobinson,\nPassport\nNumber\n780055578,\nresiding\nat\n1\nRabin\nst,\nTel\nAviv,\nIsrael,\nEmail: \njackrobinson@gmail.com\n("\nAdvisor\n").\nWhereas,\nAdvisor\nhas\nexpertise\nand/or\nknowledge\nand/or\nrelationships,\nwhich\nare\nrelevant\nto\nthe \nCompany’ s\nbusiness\nand\nthe\nCompany\nhas\nasked\nAdvisor\nto\nprovide\nit\nwith\ncertain\nAdvisory \nservices,\nas\ndescribed\nin\nthis\nAgreement;\nand', metadata={'doc_id': '7945f33c-d3ec-406b-87e4-50e92fea56e8', 'page': 0, 'source': './data/Robinson Advisory.docx.pdf'}),
 Document(page_content='ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdvisory\nServices\nAgreement\nis\nentered\ninto\nas\nof\nJune\n15\nth\n,\n2023\n(the\n“\nEffective\nDate\n”),\nby\nand \nbetween\nCloud\nInvestments\nLtd.,\nID\n51-426526-3,\nan\nIsraeli\ncompany\n(the\n"\nCompany\n"),\nand\nMr.\nJack \nRobinson,\nPassport\nNumber\n780055578,\nresiding\nat\n1\nRabin\nst,\nTel\nAviv,\nIsrael,\nEmail: \njack

In [12]:
display(Markdown(f"{sub_docs[0].page_content}"))

(the
"
Company
"),
and
Mr.
Jack 
Robinson,
Passport
Number
780055578,
residing
at
1
Rabin
st,
Tel
Aviv,
Israel,
Email: 
jackrobinson@gmail.com
("
Advisor
").
Whereas,
Advisor
has
expertise
and/or
knowledge
and/or
relationships,
which
are
relevant
to
the 
Company’ s
business
and
the
Company
has
asked
Advisor
to
provide
it
with
certain
Advisory 
services,
as
described
in
this
Agreement;
and

It contains the answer: `1 Rabin st, Tel Aviv, Israel`

In [13]:
len(sub_docs[0].page_content)

391

Let’s now retrieve from the overall retriever. This should return large documents - since it returns the documents where the smaller chunks are located.

In [14]:
retrieved_docs = retriever.get_relevant_documents("In which street does the Advisor live?")

In [15]:
len(retrieved_docs[0].page_content)

3442

In [16]:
display(Markdown(f"{retrieved_docs[0].page_content}"))

ADVISOR Y
SER VICES
AGREEMENT
This
Advisory
Services
Agreement
is
entered
into
as
of
June
15
th
,
2023
(the
“
Effective
Date
”),
by
and 
between
Cloud
Investments
Ltd.,
ID
51-426526-3,
an
Israeli
company
(the
"
Company
"),
and
Mr.
Jack 
Robinson,
Passport
Number
780055578,
residing
at
1
Rabin
st,
Tel
Aviv,
Israel,
Email: 
jackrobinson@gmail.com
("
Advisor
").
Whereas,
Advisor
has
expertise
and/or
knowledge
and/or
relationships,
which
are
relevant
to
the 
Company’ s
business
and
the
Company
has
asked
Advisor
to
provide
it
with
certain
Advisory 
services,
as
described
in
this
Agreement;
and
Whereas,
Advisor
has
agreed
to
provide
the
Company
with
such
services,
subject
to
the
terms
set
forth 
in
this
Agreement.
NOW
THEREFORE
THE
PARTIES
AGREE
AS
FOLLOWS:
1.
Services:
1.1
Advisor
shall
provide
to
the
Company ,
as
an
independent
contractor ,
software
development 
services,
and
/
or
any
other
services
as
agreed
by
the
parties
from
time
to
time
(the
“
Services
”). 
Advisor
shall
not
appoint
any
sub-agents
or
delegates
in
connection
with
performance
of
the 
Services,
nor
assign
any
of
his
obligations
or
rights
hereunder
without
obtaining
the
prior
written 
consent
of
the
Company .
Without
derogating
from
the
foregoing,
the
Advisor
shall
not
be
required
to
provide
the
Services 
for
a
period
of
eighteen
(18)
days
in
each
12-month
period.
1.2
In
performing
the
Services,
Advisor
shall
devote
his
time,
know-how ,
expertise,
talent, 
experience
and
best
efforts
and
attention
to
the
provision
of
the
Services,
as
shall
be
reasonably 
required
by
the
Company .
Advisor
shall
render
the
Services
in
a
diligent
manner
and
at
a
high 
professional
standard
for
the
benefit
of
the
Company
and
shall
perform
and
dischar ge
well
and 
faithfully ,
with
devotion,
honesty
and
fidelity ,
his
obligations
hereunder .
1.3
Advisor
further
agrees
and
undertakes
to
(i)
comply
with
all
the
Company's
policies,
procedures 
and
objectives,
as
in
effect
from
time
to
time;
(ii)
adhere
to
any
applicable
law
or
provision, 
pertaining
to
his
services;
(iii)
not
take
any
action
with
the
intention
that
the
same
will
cause 
material
damage
to
the
good
name
of
the
Company;
(iv)
keep
the
contents
of
this
Agreement 
confidential
and
not
to
disclose
the
existence
or
contents
of
this
Agreement
to
any
third
party 
without
the
prior
written
consent
of
the
Company .
2.
Time
Tracking:
Advisor
shall
provide
the
Company
with
a
written
report,
in
a
format
acceptable
by 
the
Company ,
setting
forth
the
number
of
hours
in
which
he
provided
the
Services,
on
a
daily
basis,
as 
well
as
an
aggregated
monthly
report
at
the
last
day
of
each
calendar
month.
3.
No
Conflicts:
Advisor
represents
that
there
is
no
contractual
or
other
legal
restriction
preventing
or 
limiting
him/her
from
entering
into
this
Agreement
or
performing
any
of
his/her
obligations 
hereunder .
Advisor
shall
immediately
and
without
delay
inform
the
Company
of
any
affairs
and/or 
matters
that
might
constitute
a
conflict
of
interest
with
his/her
services
to
Company
and/or
the 
interests
of
the
Company .
Advisor
agrees
to
indemnify ,
defend
and
hold
harmless
the
Company
and 
its
directors,
officers
and
employees
against
all
claims,
liabilities,
and
costs,
including
reasonable 
attorneys'
fees
in
connection
with
any
breach
by
Advisor ,
of
any
obligations
pursuant
to
a
prior
or 
existing
engagement
with
any
other
third
party ,
including
without
limitation
other
employers
or 
clients.

# Retrieving larger chunks
Sometimes, the full documents can be too big to want to retrieve them as is. In that case, what we really want to do is to first split the raw documents into larger chunks, and then split it into smaller chunks. We then index the smaller chunks, but on retrieval we retrieve the larger chunks (but still not the full documents).

In [17]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=st_embedder
)
# The storage layer for the parent documents
store = InMemoryStore()

In [18]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [19]:
retriever.add_documents(docs_test)

We can see that there are much more than four documents now - these are the larger chunks.

In [20]:
len(list(store.yield_keys()))

9

Let’s make sure the underlying vector store still retrieves the small chunks.

In [21]:
sub_docs = vectorstore.similarity_search("In which street does the Advisor live?")

In [22]:
sub_docs

[Document(page_content='(the\n"\nCompany\n"),\nand\nMr.\nJack \nRobinson,\nPassport\nNumber\n780055578,\nresiding\nat\n1\nRabin\nst,\nTel\nAviv,\nIsrael,\nEmail: \njackrobinson@gmail.com\n("\nAdvisor\n").\nWhereas,\nAdvisor\nhas\nexpertise\nand/or\nknowledge\nand/or\nrelationships,\nwhich\nare\nrelevant\nto\nthe \nCompany’ s\nbusiness\nand\nthe\nCompany\nhas\nasked\nAdvisor\nto\nprovide\nit\nwith\ncertain\nAdvisory \nservices,\nas\ndescribed\nin\nthis\nAgreement;\nand', metadata={'doc_id': '93bca945-cbdf-4b0f-8452-96f48f1dc305', 'page': 0, 'source': './data/Robinson Advisory.docx.pdf'}),
 Document(page_content='accordance\nwith\nthe\nlaws\nof\nthe\nState\nof\nIsrael,\nwithout\ngiving\neffect \nto\nits\nlaws\npertaining\nto\nconflict\nof\nlaws. \nThe\nAdvisor\nagrees\nthat\nany\nand\nall\ndisputes\nin\nconnection\nwith\nthis\nUndertaking\nshall\nbe \nsubmitted\nto\nthe\nexclusive\njurisdiction\nof\nthe\ncompetent\ncourts\nin\nthe\ncity\nof\nTel\nAviv-Y afo,\nIsrael.\n9.\nPre-Undertaking

In [23]:
display(Markdown(f"{sub_docs[0].page_content}"))

(the
"
Company
"),
and
Mr.
Jack 
Robinson,
Passport
Number
780055578,
residing
at
1
Rabin
st,
Tel
Aviv,
Israel,
Email: 
jackrobinson@gmail.com
("
Advisor
").
Whereas,
Advisor
has
expertise
and/or
knowledge
and/or
relationships,
which
are
relevant
to
the 
Company’ s
business
and
the
Company
has
asked
Advisor
to
provide
it
with
certain
Advisory 
services,
as
described
in
this
Agreement;
and

In [24]:
retrieved_docs = retriever.get_relevant_documents("In which street does the Advisor live?")
retrieved_docs

[Document(page_content='ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdvisory\nServices\nAgreement\nis\nentered\ninto\nas\nof\nJune\n15\nth\n,\n2023\n(the\n“\nEffective\nDate\n”),\nby\nand \nbetween\nCloud\nInvestments\nLtd.,\nID\n51-426526-3,\nan\nIsraeli\ncompany\n(the\n"\nCompany\n"),\nand\nMr.\nJack \nRobinson,\nPassport\nNumber\n780055578,\nresiding\nat\n1\nRabin\nst,\nTel\nAviv,\nIsrael,\nEmail: \njackrobinson@gmail.com\n("\nAdvisor\n").\nWhereas,\nAdvisor\nhas\nexpertise\nand/or\nknowledge\nand/or\nrelationships,\nwhich\nare\nrelevant\nto\nthe \nCompany’ s\nbusiness\nand\nthe\nCompany\nhas\nasked\nAdvisor\nto\nprovide\nit\nwith\ncertain\nAdvisory \nservices,\nas\ndescribed\nin\nthis\nAgreement;\nand\nWhereas,\nAdvisor\nhas\nagreed\nto\nprovide\nthe\nCompany\nwith\nsuch\nservices,\nsubject\nto\nthe\nterms\nset\nforth \nin\nthis\nAgreement.\nNOW\nTHEREFORE\nTHE\nPARTIES\nAGREE\nAS\nFOLLOWS:\n1.\nServices:\n1.1\nAdvisor\nshall\nprovide\nto\nthe\nCompany ,\nas\nan\nindependent\ncontractor

In [25]:
display(Markdown(f"{retrieved_docs[0].page_content}"))

ADVISOR Y
SER VICES
AGREEMENT
This
Advisory
Services
Agreement
is
entered
into
as
of
June
15
th
,
2023
(the
“
Effective
Date
”),
by
and 
between
Cloud
Investments
Ltd.,
ID
51-426526-3,
an
Israeli
company
(the
"
Company
"),
and
Mr.
Jack 
Robinson,
Passport
Number
780055578,
residing
at
1
Rabin
st,
Tel
Aviv,
Israel,
Email: 
jackrobinson@gmail.com
("
Advisor
").
Whereas,
Advisor
has
expertise
and/or
knowledge
and/or
relationships,
which
are
relevant
to
the 
Company’ s
business
and
the
Company
has
asked
Advisor
to
provide
it
with
certain
Advisory 
services,
as
described
in
this
Agreement;
and
Whereas,
Advisor
has
agreed
to
provide
the
Company
with
such
services,
subject
to
the
terms
set
forth 
in
this
Agreement.
NOW
THEREFORE
THE
PARTIES
AGREE
AS
FOLLOWS:
1.
Services:
1.1
Advisor
shall
provide
to
the
Company ,
as
an
independent
contractor ,
software
development 
services,
and
/
or
any
other
services
as
agreed
by
the
parties
from
time
to
time
(the
“
Services
”). 
Advisor
shall
not
appoint
any
sub-agents
or
delegates
in
connection
with
performance
of
the 
Services,
nor
assign
any
of
his
obligations
or
rights
hereunder
without
obtaining
the
prior
written 
consent
of
the
Company .
Without
derogating
from
the
foregoing,
the
Advisor
shall
not
be
required
to
provide
the
Services 
for
a
period
of
eighteen
(18)
days
in
each
12-month
period.
1.2
In
performing
the
Services,
Advisor
shall
devote
his
time,
know-how ,
expertise,
talent, 
experience
and
best
efforts
and
attention
to
the
provision
of
the
Services,
as
shall
be
reasonably 
required
by
the
Company .
Advisor
shall
render
the
Services
in
a
diligent
manner
and
at
a
high 
professional
standard
for
the
benefit
of
the
Company
and
shall
perform
and
dischar ge
well
and 
faithfully ,
with
devotion,
honesty
and
fidelity ,
his
obligations
hereunder .
1.3
Advisor
further
agrees
and
undertakes
to
(i)
comply
with
all
the
Company's
policies,
procedures 
and
objectives,
as
in
effect
from
time
to
time;
(ii)
adhere
to
any
applicable
law
or

In [26]:
len(retrieved_docs[0].page_content)

2000

In [27]:
from IPython.display import display, Markdown
from dotenv import load_dotenv, find_dotenv
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import chromadb
from langchain_community.vectorstores import Chroma
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langchain_community.embeddings import SentenceTransformerEmbeddings


class AdvancedRetriever:
    def __init__(self):
        load_dotenv(find_dotenv())

    
    def parent_documents_retriever(self, loaded_docs):
        # This text splitter is used to create the parent documents
        parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
        # This text splitter is used to create the child documents
        # It should create documents smaller than the parent
        child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
        # The vectorstore to use to index the child chunks
        vectorstore = Chroma(
            collection_name="split_parents", embedding_function=SentenceTransformerEmbeddings()
        )
        # The storage layer for the parent documents
        store = InMemoryStore()

        retriever = ParentDocumentRetriever(
            vectorstore=vectorstore,
            docstore=store,
            child_splitter=child_splitter,
            parent_splitter=parent_splitter,
        )

        retriever.add_documents(loaded_docs)

        return vectorstore, retriever
    
  

In [28]:
# from utils.Advanced_Retriever import AdvancedRetriever

advanced_retriver = AdvancedRetriever()
vectorstore1, retriever1 = advanced_retriver.parent_documents_retriever(docs_test)

In [29]:
vectorstore1.similarity_search("In which street does the Advisor live?")[0].page_content

'(the\n"\nCompany\n"),\nand\nMr.\nJack \nRobinson,\nPassport\nNumber\n780055578,\nresiding\nat\n1\nRabin\nst,\nTel\nAviv,\nIsrael,\nEmail: \njackrobinson@gmail.com\n("\nAdvisor\n").\nWhereas,\nAdvisor\nhas\nexpertise\nand/or\nknowledge\nand/or\nrelationships,\nwhich\nare\nrelevant\nto\nthe \nCompany’ s\nbusiness\nand\nthe\nCompany\nhas\nasked\nAdvisor\nto\nprovide\nit\nwith\ncertain\nAdvisory \nservices,\nas\ndescribed\nin\nthis\nAgreement;\nand'

In [30]:
retriever1.get_relevant_documents("In which street does the Advisor live?")

[Document(page_content='ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdvisory\nServices\nAgreement\nis\nentered\ninto\nas\nof\nJune\n15\nth\n,\n2023\n(the\n“\nEffective\nDate\n”),\nby\nand \nbetween\nCloud\nInvestments\nLtd.,\nID\n51-426526-3,\nan\nIsraeli\ncompany\n(the\n"\nCompany\n"),\nand\nMr.\nJack \nRobinson,\nPassport\nNumber\n780055578,\nresiding\nat\n1\nRabin\nst,\nTel\nAviv,\nIsrael,\nEmail: \njackrobinson@gmail.com\n("\nAdvisor\n").\nWhereas,\nAdvisor\nhas\nexpertise\nand/or\nknowledge\nand/or\nrelationships,\nwhich\nare\nrelevant\nto\nthe \nCompany’ s\nbusiness\nand\nthe\nCompany\nhas\nasked\nAdvisor\nto\nprovide\nit\nwith\ncertain\nAdvisory \nservices,\nas\ndescribed\nin\nthis\nAgreement;\nand\nWhereas,\nAdvisor\nhas\nagreed\nto\nprovide\nthe\nCompany\nwith\nsuch\nservices,\nsubject\nto\nthe\nterms\nset\nforth \nin\nthis\nAgreement.\nNOW\nTHEREFORE\nTHE\nPARTIES\nAGREE\nAS\nFOLLOWS:\n1.\nServices:\n1.1\nAdvisor\nshall\nprovide\nto\nthe\nCompany ,\nas\nan\nindependent\ncontractor

# Rag Pipeline

In [31]:
from langchain import OpenAI, VectorDBQA
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain_community.llms import OpenAI

In [32]:
def conversation_chain(vectorstore):
    llm = ChatOpenAI(temperature=0.0)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    conversation_chain= ConversationalRetrievalChain.from_llm(
        llm = llm,
        memory = memory,
        retriever = vectorstore.as_retriever(),
        )
    return conversation_chain

In [36]:
import os
load_dotenv(find_dotenv())


True

In [34]:
rag_chain = conversation_chain(vectorstore1)
rag_chain.invoke("In which street does the Advisor live?")

/home/crs/10Academy/w11/Contract-Advisor-RAG/w11_rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


{'question': 'In which street does the Advisor live?',
 'chat_history': [HumanMessage(content='In which street does the Advisor live?'),
  AIMessage(content='The Advisor lives at 1 Rabin Street, Tel Aviv, Israel.')],
 'answer': 'The Advisor lives at 1 Rabin Street, Tel Aviv, Israel.'}

In [37]:
def conversation_chain_1(retriever):
    llm = ChatOpenAI(temperature=0.0)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    conversation_chain= ConversationalRetrievalChain.from_llm(
        llm = llm,
        memory = memory,
        retriever = retriever,
        )
    return conversation_chain

In [38]:
rag_chain_1 = conversation_chain_1(retriever1)
rag_chain_1.invoke("In which street does the Advisor live?")

{'question': 'In which street does the Advisor live?',
 'chat_history': [HumanMessage(content='In which street does the Advisor live?'),
  AIMessage(content='The Advisor, Jack Robinson, lives at 1 Rabin St, Tel Aviv, Israel.')],
 'answer': 'The Advisor, Jack Robinson, lives at 1 Rabin St, Tel Aviv, Israel.'}

In [35]:
# def qNa_chain(vectorsotre):
#     llm = ChatOpenAI(temperature=0.0)
#     qa_stuff = RetrievalQA.from_chain_type(
#         llm = llm,
#         chain_type = "stuff",
#         retriever = vectorsotre.as_retriever(),
#         verbose = False,
#     )
#     return qa_stuff